In [1]:
# IMPORT REQUIRED LIBRARIES

import os

import numpy as np
import pandas as pd

SEED = 9

In [2]:
# IMPORT RAW DATA

spectra_path_1 = "C:\\Users\\argan\\OneDrive\\Desktop\\DDP\\Data 2\\Spectra 1"
spectra_path_2 = "C:\\Users\\argan\\OneDrive\\Desktop\\DDP\\Data 2\\Spectra 2"
spectra_path_3 = "C:\\Users\\argan\\OneDrive\\Desktop\\DDP\\Data 2\\NewAppleFiles"

brix_vals_path = "C:\\Users\\argan\\OneDrive\\Desktop\\DDP\\Data 2\\BrixVal3.xlsx"
brix_vals_path_3 = "C:\\Users\\argan\\OneDrive\\Desktop\\DDP\\Data 2\\NewBrixValues.xlsx"

spectra_files_1 = os.listdir(spectra_path_1)
spectra_files_2 = os.listdir(spectra_path_2)
spectra_files_3 = os.listdir(spectra_path_3)

brix_vals = pd.read_excel(brix_vals_path, index_col="File")
brix_vals_3 = pd.read_excel(brix_vals_path_3, index_col="File")

In [3]:
# Reference Spectrum

r_spectrum = pd.read_csv(spectra_path_1+"\\"+"WhiteD1.csv")
r_spectrum = r_spectrum.mean().values

In [4]:
# Averaging intensity readings

spectra_1 = []
brix_1 = []
names_1 = []
groups_1 = []

sample_names = set(brix_vals.index)

for file in spectra_files_1:
    sample_name = file.split('.')[0]

    # Discard file if brix value not found.
    if sample_name.lower() not in sample_names:
        continue

    brix_val, group = brix_vals.loc[sample_name.lower()]

    # Discard file if brix value type is invalid.
    if isinstance(brix_val, str) or np.isnan(brix_val):
        continue
        
    spectrum = pd.read_csv(spectra_path_1+"\\"+file, header=None, skiprows=2)
    
    if spectrum.shape[1] == 19:
        spectrum = spectrum.iloc[:, :-1]

    spectrum = spectrum.mean().values

    names_1.append(sample_name)
    spectra_1.append(spectrum)
    brix_1.append(float(brix_val))
    groups_1.append(group)

wavelengths = pd.read_csv(spectra_path_1+"\\"+file).columns

In [5]:
# Averaging intensity readings

spectra_2 = []
brix_2 = []
names_2 = []
groups_2 = []

for file in spectra_files_2:
    sample_name = file.split('.')[0]

    # Discard file if brix value not found.
    if sample_name.lower() not in sample_names:
        continue

    brix_val, group = brix_vals.loc[sample_name.lower()]
    
    # Discard file if brix value type is invalid.
    if isinstance(brix_val, str) or np.isnan(brix_val):
        continue
        
    spectrum = pd.read_csv(spectra_path_2+"\\"+file, header=None, skiprows=2)
    
    if spectrum.shape[1] == 19:
        spectrum = spectrum.iloc[:, :-1]

    spectrum = spectrum.mean().values

    names_2.append(sample_name)
    spectra_2.append(spectrum)
    brix_2.append(float(brix_val))
    groups_2.append(group)

In [6]:
# Averaging intensity readings

spectra_3 = []
brix_3 = []
names_3 = []
groups_3 = []

sample_names = set(brix_vals_3.index)

for file in spectra_files_3:
    sample_name = file.split('.')[0]

    # Discard file if brix value not found.
    if sample_name.lower() not in sample_names:
        continue

    brix_val, group = brix_vals_3.loc[sample_name.lower()]
    
    # Discard file if brix value type is invalid.
    if isinstance(brix_val, str) or np.isnan(brix_val):
        continue
        
    spectrum = pd.read_csv(spectra_path_3+"\\"+file, header=None, skiprows=2)
    
    if spectrum.shape[1] == 19:
        spectrum = spectrum.iloc[:, :-1]

    spectrum = spectrum.mean().values

    names_3.append(sample_name)
    spectra_3.append(spectrum)
    brix_3.append(float(brix_val))
    groups_3.append(group)

In [7]:
# Calculating absorbance values
abs_spectra = []
for i in range(len(spectra_1)):
    abs_spectra.append(-np.log10(np.divide(spectra_1[i], r_spectrum)))

abs_df_1 = pd.DataFrame(abs_spectra, columns=wavelengths)
abs_df_1["Name"] = names_1
abs_df_1["Brix Values"] = brix_1
abs_df_1["Group"] = groups_1

abs_df_1.shape

(46, 21)

In [8]:
# Calculating absorbance values
abs_spectra = []
for i in range(len(spectra_2)):
    abs_spectra.append(-np.log10(np.divide(spectra_2[i], r_spectrum)))

abs_df_2 = pd.DataFrame(abs_spectra, columns=wavelengths)
abs_df_2["Name"] = names_2
abs_df_2["Brix Values"] = brix_2
abs_df_2["Group"] = groups_2

abs_df_2.shape

(245, 21)

In [9]:
# Calculating absorbance values
abs_spectra = []
for i in range(len(spectra_3)):
    abs_spectra.append(-np.log10(np.divide(spectra_3[i], r_spectrum)))

abs_df_3 = pd.DataFrame(abs_spectra, columns=wavelengths)
abs_df_3["Name"] = names_3
abs_df_3["Brix Values"] = brix_3
abs_df_3["Group"] = groups_3

abs_df_3.shape

(96, 21)

In [11]:
# Removing outliers - Negative absorbance values and extreme brix values

abs_df_noisy = pd.concat([abs_df_1, abs_df_2, abs_df_3], ignore_index=True)

abs_df_noisy = abs_df_noisy[abs_df_noisy['900 nm'] > 0]
abs_df_noisy = abs_df_noisy[abs_df_noisy['730 nm'] > 0]
abs_df_noisy = abs_df_noisy[abs_df_noisy['Brix Values'] < 18]

abs_df_noisy.reset_index(inplace=True, drop=True)

In [12]:
abs_df_noisy.to_csv("C:\\Users\\argan\\OneDrive\\Desktop\\DDP\\Final Data\\NoisyAbs.csv")

In [22]:
abs_df_noisy.shape

(369, 21)

### Denoising - Sample Averaging

In [14]:
abs_df_1.drop(columns=['Name'], inplace=True)
abs_df_2.drop(columns=['Name'], inplace=True)
abs_df_3.drop(columns=['Name'], inplace=True)

In [15]:
# Removing outliers - Negative absorbance values and extreme brix values

abs_df_1 = abs_df_1[abs_df_1['900 nm'] > 0]
abs_df_1 = abs_df_1[abs_df_1['730 nm'] > 0]
abs_df_1 = abs_df_1[abs_df_1['Brix Values'] < 18]
abs_df_1.reset_index(inplace=True, drop=True)

# Sample averaging

abs_df_mean_grp_1 = abs_df_1.groupby(['Group']).mean()

abs_df_mean_1 = abs_df_mean_grp_1.reset_index(drop=True)

abs_df_mean_1.shape

(11, 19)

In [16]:
# Removing outliers - Negative absorbance values and extreme brix values

abs_df_2 = abs_df_2[abs_df_2['900 nm'] > 0]
abs_df_2 = abs_df_2[abs_df_2['730 nm'] > 0]
abs_df_2 = abs_df_2[abs_df_2['Brix Values'] < 18]
abs_df_2.reset_index(inplace=True, drop=True)

# Sample averaging

abs_df_mean_grp_2 = abs_df_2.groupby(['Group']).mean()

abs_df_mean_2 = abs_df_mean_grp_2.reset_index(drop=True)

abs_df_mean_2.shape

(64, 19)

In [17]:
# Removing outliers - Negative absorbance values and extreme brix values

abs_df_3 = abs_df_3[abs_df_3['900 nm'] > 0]
abs_df_3 = abs_df_3[abs_df_3['730 nm'] > 0]
abs_df_3 = abs_df_3[abs_df_3['Brix Values'] < 18]
abs_df_3.reset_index(inplace=True, drop=True)

# Sample averaging

abs_df_mean_grp_3 = abs_df_3.groupby(['Group']).mean()

abs_df_mean_3 = abs_df_mean_grp_3.reset_index(drop=True)

abs_df_mean_3.shape

(24, 19)

In [18]:
# Merging

abs_df = pd.concat([abs_df_mean_1, abs_df_mean_2, abs_df_mean_3], ignore_index=True)

In [19]:
abs_df.to_csv("C:\\Users\\argan\\OneDrive\\Desktop\\DDP\\Final Data\\AvgAbs.csv")